In [ ]:
from datasets import load_dataset

data = load_dataset("wikipedia", "20220301.en", trust_remote_code=True)

In [ ]:
!rm -rf /root/.cache/huggingface

In [ ]:
c = ""
a = data["train"].select(range(10000))  
for j, i in enumerate(a):
        c += i["text"] + " "

In [ ]:
len(c)

In [ ]:
with open("a.txt", "w") as f:
    f.write(c)
    f.close()

In [ ]:
!mkdir vocab
!mkdir train

In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
special_tokens = ["<PAD>", "<SEP>", "<UNK>", "<SOS>", "<EOS>"]
tokenizer.train(files="a.txt", vocab_size=50000, special_tokens=special_tokens)
tokenizer.save_model("/kaggle/working/vocab")

In [ ]:
!rm a.txt

In [ ]:
from tqdm import tqdm
import torch
import numpy as np
batch_size = 10000
num_batches = len(data["train"]) // batch_size

for j in tqdm(range(0, 1500000, batch_size), desc="Processing Batches"):
    batch_texts = [i["text"] for i in data["train"].select(range(j, j + batch_size))]
    encoded_batch = tokenizer.encode_batch(batch_texts)
    batch_arrays = []
    for enc in encoded_batch:
        batch_arrays.extend(enc.ids)  
        batch_arrays.append(1) 
    np.savez_compressed(f"train/batch_{j//batch_size}.npz", batch_arrays=np.array(batch_arrays, dtype=np.int32))    
    print(f"Processed {j + len(batch_texts)} samples")